In [2]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import tqdm
import glob
import json

# Coleta 
Obtenho informações salvas anteriormente em um arquivo Json

In [3]:
df = pd.read_json("parsed_videos.json", lines=True)
df.head()

,link,title,query
0,/watch?v=EuQBI7DC-f0,Machine Learning para Competições Kaggle - Cur...,machine+learning
1,/watch?v=z-dLiGxbTkU,Machine Learning using Python - Project Assign...,machine+learning
2,/watch?v=BL0fKwb-6G0,Minicurso de Introdução à Machine Learning e I...,machine+learning
3,/watch?v=PjiCKtkLWPA,Machine Learning From Stanford University | Co...,machine+learning
4,/watch?v=7IEl0PCizyI,Categorical Plots | Machine Learning | Intellify,machine+learning


<br>Obtenho apenas a lista de links. 
<br> (únicos pois ao pesquisar machine learning e data science, pode-se ter sugerido o mesmo vídeo)

In [4]:
lista_de_links = df['link'].unique()
len(lista_de_links)

242

Função comentada linha a linha

In [4]:
# crio uma string generalizando a parte padrão dos links
url = "https://www.youtube.com{link}"
# percorro a lista de links criado anteriormente
for link in lista_de_links:
    # adiciono os links da lista a string padrão, para obtermos o link de acesso como mostrado abaixo
    urll = url.format(link=link)
    print(urll)
    # obtemos as informações da página, dando um request nela
    response = rq.get(urll)
    # observamos que a parte "/watch?v=" é igual para todos, então consideramos o nome só o final do link
    link_name = re.search("v=(.*)", link).group(1)
    # criamos arquivos do código fonte da páginas do vídeos.
    # (anteriormente foi das páginas de busca, agora é da página onde assistimos o vídeo)
    with open("./dados_brutos/video_{}.html".format(link_name), "w+", encoding="utf-8") as output:
        output.write(response.text)
    time.sleep(2)
    

https://www.youtube.com/watch?v=EuQBI7DC-f0
https://www.youtube.com/watch?v=z-dLiGxbTkU
https://www.youtube.com/watch?v=BL0fKwb-6G0
https://www.youtube.com/watch?v=PjiCKtkLWPA
https://www.youtube.com/watch?v=7IEl0PCizyI
https://www.youtube.com/watch?v=iRhDvk3Hf-E
https://www.youtube.com/watch?v=LUnDYIIX5DE
https://www.youtube.com/watch?v=puLQBYgrDqE
https://www.youtube.com/watch?v=t1d_if_yA2k
https://www.youtube.com/watch?v=2OgBxYCnVGk
https://www.youtube.com/watch?v=J6A_ieEG7oo
https://www.youtube.com/watch?v=vEwjvJjN8ZA
https://www.youtube.com/watch?v=f3y8A8fazcE
https://www.youtube.com/watch?v=FmkXANqeYng
https://www.youtube.com/watch?v=RvH-XGCNc90
https://www.youtube.com/watch?v=-HDESJBanZg
https://www.youtube.com/watch?v=y_1ljfOMtLE
https://www.youtube.com/watch?v=2d_tTVr2lDk
https://www.youtube.com/watch?v=oHBlX7nZ_eA
https://www.youtube.com/watch?v=EA12Tjd8XZM
https://www.youtube.com/watch?v=Bf4VvGgU5ps
https://www.youtube.com/watch?v=5cN36C5WgGE
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=YTxH8DpVeno
https://www.youtube.com/watch?v=jbVF4G_Bilg
https://www.youtube.com/watch?v=VITk3IOxG_E
https://www.youtube.com/watch?v=S6GVXk6kbcs
https://www.youtube.com/watch?v=qJlYzKn7my0
https://www.youtube.com/watch?v=sxoA7zviiqQ
https://www.youtube.com/watch?v=uVHjY7owVl0
https://www.youtube.com/watch?v=rtzUHuGOZ7o
https://www.youtube.com/watch?v=XmmR1494fa8
https://www.youtube.com/watch?v=IraU2xyAoKc
https://www.youtube.com/watch?v=Q0_Xajic_9U
https://www.youtube.com/watch?v=7FwITPrBvLI
https://www.youtube.com/watch?v=XY8-t6PU85U
https://www.youtube.com/watch?v=8Rt5FTD6PQg
https://www.youtube.com/watch?v=avkwtsyziXg
https://www.youtube.com/watch?v=GDza1aHcpb4
https://www.youtube.com/watch?v=tvzWEXZFSSA
https://www.youtube.com/watch?v=uIDtrrEtaeQ
https://www.youtube.com/watch?v=K_G8GPXnouI
https://www.youtube.com/watch?v=K5KDZLHsr1o
https://www.youtube.com/watch?v=PHQt0aUasRg
https://www.youtube.com/watch?v=_DXfuI5DG2k
https://www.youtube.com/watch?v=

# Processamento dos dados

In [5]:
with open("parsed_video_info.json", "w+", encoding="utf-8") as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/video*"))):
        with open(video_file, "r+", encoding="utf-8") as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, "html.parser")
            
            # encontrar toda tag 'class' do arquivo cujo dentro dela tenha watch
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            # encontrar toda tag 'id' do arquivo cujo dentro dela tenha watch
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            # obter o link do canal, para análise
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            # tem algumas informações sobre o vídeo
            meta = parsed.find_all("meta")
            
            data = dict()
            
            for e in class_watch:
                # criando o nome da chave no dicionário, que será o join das strings dentro de "class"
                colname = "_".join(e["class"])
                # se tiver clearfix dentro da tag "class", não me interessa
                if "clearfix" in colname:
                    continue
                #strip = tira espaços em branco do começo e fim da string
                data[colname] = e.text.strip()
            
            for e in id_watch:
                # informação que está dentro da tag 'id', vira chave do dicionário
                colname = e["id"]
                # textos dentro da tag, são os valores do dic
                data[colname] = e.text.strip()
                
            for e in meta:
                # a chave do dic vai ser a tag 'property'
                colname = e.get("property")
                if colname is not None:
                    # o valor vai ser a tag 'content'
                    data[colname] = e["content"]
            # enumerate = intera lista        
            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e["href"]
                
            output.write("{}\n".format(json.dumps(data)))
                

C:\Users\Guilherme\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


# Verificação
Informações obtidas anteriormente é colocada em um DataFrame

In [6]:
df = pd.read_json("parsed_video_info.json", lines= True)
df

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,...,channel_link_7,channel_link_8,channel_link_9,channel_link_10,channel_link_11,channel_link_12,watch-title_metadata-updateable-title,watch-view-count_metadata-updateable-viewership,watch-time-text_metadata-updateable-date-text,content-alignment_watch-small_off-screen-trigger
0,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# descobrindo o número de linhas e colunas
df.shape

(242, 131)

In [8]:
# visualizando TODAS as colunas do DataFrame
pd.set_option("display.max_columns", 131)
df.head(1)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch8-sentiment-actions,watch7-views-info,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,channel_link_1,channel_link_2,channel_link_3,yt-pl-watch-queue-overlay,watch-channel-brand-div,watch-channel-brand-div-text,channel_link_4,watch-skeleton,watch-page-skeleton,watch-sidebar-live-chat,watch-meta-item,watch-sidebar-discussion,channel_link_5,channel_link_6,channel_link_7,channel_link_8,channel_link_9,channel_link_10,channel_link_11,channel_link_12,watch-title_metadata-updateable-title,watch-view-count_metadata-updateable-viewership,watch-time-text_metadata-updateable-date-text,content-alignment_watch-small_off-screen-trigger
0,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de

## Selecionando colunas que achamos importante

In [10]:
colunas_selecionadas = [
"watch-title", "watch-view-count", "watch-time-text", "content_watch-info-tag-list", "watch7-headline",
"watch7-user-header", "watch8-sentiment-actions", "og:image", "og:image:width", "og:image:height", 
"og:description", "og:video:width", "og:video:height", "og:video:tag", "channel_link_0" 
]

In [12]:
# mostrando DataFrame apenas com colunas selecionadas
df[colunas_selecionadas].head()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Binary Image Classification with CNN (Deep Lea...,244 visualizações,Publicado em 23 de mar. de 2020,Ciência e tecnologia,#download #Kaggle #Google\n\n\n\n Binary Im...,TheClassofAI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,244 visualizações\n\n\n\n\n\n\n\n6\n\nGostou d...,https://i.ytimg.com/vi/-DDrE6T0ct4/hqdefault.jpg,480.0,360.0,"In this tutorial, we have covered how to train...",1280.0,720.0,ImageClassification,/channel/UCYlOdJBJQN4c7k25uzwSwJA
1,Azure Machine Learning in Action! | COM216,Nenhuma visualização,Publicado em 26 de mai. de 2020,Ciência e tecnologia,Azure Machine Learning in Action! | COM216,Microsoft Developer\n\n\n\n\n\n\n\n\n\n\n\n\n\...,Nenhuma visualização\n\n\n\n\n\n\n\n0\n\nGosto...,https://i.ytimg.com/vi/-HDESJBanZg/maxresdefau...,1280.0,720.0,Community Connections experience - What is Mac...,1280.0,720.0,Lindsey Murphy,/channel/UCsMica-v34Irf9KVTh6xx-g
2,Tutorial 57: Bellman Equation and Q Learning i...,60 visualizações,Publicado em 24 de mai. de 2020,Ciência e tecnologia,#deeplearning #machineleanring #artificialinte...,Fahad Hussain\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,60 visualizações\n\n\n\n\n\n\n\n6\n\nGostou de...,https://i.ytimg.com/vi/-cmWl4g4oyY/hqdefault.jpg,480.0,360.0,"For Code, Slides and Notes https://fahadhussai...",1280.0,720.0,Fahad,/channel/UCapJpINJKHzflWwCQ8Kse2g
3,Introduction to Deep Learning - 6.Training Neu...,559 visualizações,Publicado em 23 de mai. de 2020,Ciência e tecnologia,Introduction to Deep Learning - 6.Training Neu...,Matthias Niessner\n\n\n\n\n\n\n\n\n\n\n\n\n\nC...,559 visualizações\n\n\n\n\n\n\n\n22\n\nGostou ...,https://i.ytimg.com/vi/-l_f5TeG_fo/hqdefault.jpg,480.0,360.0,Website: https://niessner.github.io/I2DL/ Slid...,960.0,720.0,machinelearning,/channel/UCXN2nYjVT0cR9G61RPEzK5Q
4,Intro to Machine Learning - Missing Values,36 visualizações,Publicado em 24 de mai. de 2020,Educação,Intro to Machine Learning - Missing Values,Sonu Jha\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando...,36 visualizações\n\n\n\n\n\n\n\n6\n\nGostou de...,https://i.ytimg.com/vi/-vgTkKTHB7g/maxresdefau...,1280.0,720.0,"In this tutorial, you will learn three approac...",960.0,720.0,#machine#learning,/channel/UCjEZLTw9tJHKxo_lAeNGpuA


In [13]:
# salvamos DataFrame (com colunas selecionadas) em um .feather
df[colunas_selecionadas].to_feather("raw_data.feather")

In [14]:
# salvamos DataFrame (com colunas selecionadas) em um .csv
df[colunas_selecionadas].to_csv("raw_data_sem_labels.csv")